In [ ]:
%pip install accelerate  ## for use of device map feature
%pip install transformers
#%pip install langchain==0.0.199
%pip install langchain==0.1.16
#%pip install chromadb==0.3.29
%pip install chromadb==0.4.24
%pip install fastapi==0.110.1
%pip install pypdf
%pip install xformers
%pip install sentence_transformers==2.2.2  ## this version to avoid bug in 2.2.3
%pip install InstructorEmbedding
%pip install pycryptodome
%pip install cython cchardet
#%pip install gradio==3.48.0
%pip install gradio ### need updated gradio for API, which needs upgraded pydantic and fastapi
#%pip install pydantic==1.10
%pip install pydantic==2.7.0

In [ ]:
import torch
# import sys
# import os
# from subprocess import call
# print('_____Python, Pytorch, Cuda info____')
# print('__Python VERSION:', sys.version)
# print('__pyTorch VERSION:', torch.__version__)
# print('__CUDA RUNTIME API VERSION')
# #os.system('nvcc --version')
# print('__CUDNN VERSION:', torch.backends.cudnn.version())
# print('_____nvidia-smi GPU details____')
# call(["nvidia-smi", "--format=csv", "--query-gpu=index,name,driver_version,memory.total,memory.used,memory.free"])
# print('_____Device assignments____')
# print('Number CUDA Devices:', torch.cuda.device_count())
# print ('Current cuda device: ', torch.cuda.current_device(), ' **May not correspond to nvidia-smi ID above, check visibility parameter')
# print("Device name: ", torch.cuda.get_device_name(torch.cuda.current_device()))

In [4]:
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
print(DEVICE)

cuda


In [5]:
from langchain import HuggingFacePipeline, PromptTemplate

### import loaders
from langchain.document_loaders import PyPDFDirectoryLoader
from langchain_community.document_loaders import PyPDFLoader
from langchain.document_loaders import CSVLoader
from langchain_community.document_loaders import UnstructuredPowerPointLoader

### for embedding
from langchain.embeddings import HuggingFaceInstructEmbeddings
from langchain.text_splitter import CharacterTextSplitter

#from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma,FAISS

### for langchain chain
from langchain.memory import ConversationBufferWindowMemory
from langchain.chains import RetrievalQA
#from langchain.chains import ConversationalRetrievalChain
from transformers import AutoTokenizer, pipeline, TextIteratorStreamer, AutoModelForCausalLM

### status bars and UI and other accessories
from tqdm import tqdm
import time
import gradio as gr
import json

2024-05-27 07:01:32.069855: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [6]:
embeddings = HuggingFaceInstructEmbeddings(
    model_name="jhgan/ko-sroberta-multitask", model_kwargs={"device": DEVICE}
)

load INSTRUCTOR_Transformer
max_seq_length  512


In [7]:
# text_splitter = RecursiveCharacterTextSplitter(chunk_size=512, chunk_overlap=32)

In [8]:
# # ### Load vector db if you've already created it --- comment this out and uncomment the above loader, splitter cells to create new vector db

# vectordb = Chroma(persist_directory="./db", embedding_function=embeddings)

In [9]:
# model_id = "meta-llama/Llama-2-7b-chat-hf"
# model = AutoModelForCausalLM.from_pretrained(model_id, torch_dtype=torch.float16, device_map="cuda")

model_id = "seongs/maymust_mistral_v1.1"
model = AutoModelForCausalLM.from_pretrained(model_id, torch_dtype=torch.float16, device_map="cuda")

tokenizer = AutoTokenizer.from_pretrained(model_id)
tokenizer.use_default_system_prompt = False

Loading checkpoint shards:   0%|          | 0/6 [00:00<?, ?it/s]

In [10]:
MAX_MAX_NEW_TOKENS = 2048
DEFAULT_MAX_NEW_TOKENS = 1024
#MAX_INPUT_TOKEN_LENGTH = int(os.getenv("MAX_INPUT_TOKEN_LENGTH", "4096"))

In [11]:
####### MEMORY PARAMETERS ###########

memory = ConversationBufferWindowMemory(
    k=5, ## number of interactions to keep in memory
    memory_key="chat_history",
    return_messages=True,  ## formats the chat_history into HumanMessage and AImessage entity list
    input_key="query",   ### for straight retrievalQA chain
    output_key="result"   ### for straight retrievalQA chain

)

In [12]:
db = None
response = None
global_question = None

In [13]:
## get list of all file URLs in vector db
from langchain_community.document_loaders.merge import MergedDataLoader

def upload_file(file):
    global db
    pdf_dir_loader = PyPDFLoader(file)
    docs = pdf_dir_loader.load()
    text_splitter = CharacterTextSplitter(chunk_size=512, chunk_overlap=100)
    texts = text_splitter.split_documents(docs)
    db = Chroma.from_documents(texts, embeddings, persist_directory="db1")
    return "파일 업로드 및 처리 완료"

In [14]:
def process_input(question, chat_history, rag_flag, system_prompt, source_docs, max_new_tokens, temperature, top_p, top_k, repetition_penalty):  # Ensure vectordb is globally accessible
    global db
    global response
    global global_question
    global_question = question
    prompt_template = "\n\n [INST] <<SYS>>" + system_prompt + "<</SYS>>\n\n Context: {context} \n\n  Question: {question} \n\n[/INST]".strip()
    PROMPT = PromptTemplate(template=prompt_template, input_variables=["context", "question"])
    print(PROMPT)
    streamer = TextIteratorStreamer(tokenizer, timeout=10.0, skip_prompt=True, skip_special_tokens=True)
    
    text_pipeline = pipeline(
        "text-generation",
        model=model,
        tokenizer=tokenizer,
        do_sample=True,
        streamer=streamer,
        max_new_tokens=max_new_tokens,
        top_p=top_p,
        top_k=top_k,
        temperature=temperature,
        repetition_penalty=repetition_penalty,
    )
    
    llm = HuggingFacePipeline(pipeline=text_pipeline)
    
    if rag_flag:
        qa_chain = RetrievalQA.from_chain_type(
            llm=llm,
            chain_type="stuff",
            chain_type_kwargs={"prompt": PROMPT},
            retriever=db.as_retriever(search_type="similarity", search_kwargs={"k": source_docs}),
            memory=memory,
            verbose=True,
            return_source_documents=True,
        )
        
        response = qa_chain({"query": question})
        outputs = ["Rag_Flag: "]
        for text in streamer:
            outputs.append(text)
            time.sleep(0.05)
            yield "".join(outputs)
    else:
        inputs = PROMPT.format(context="", question=question)
        response = llm.invoke(inputs)  # Use invoke instead of __call__ to avoid deprecation warning
        print(response)
        outputs = ["No Rag_Flag: "]
        for text in streamer:
            outputs.append(text)
            time.sleep(0.05)
            yield "".join(outputs)
    
    return response

In [15]:
def get_sources():
    global global_question
    global db

    # DB에서 유사도 검색 결과를 가져옴
    docs = db.similarity_search_with_score(global_question)
    
    # 결과를 저장할 리스트 초기화
    results = []

    # 모든 문서와 유사도 점수를 리스트에 저장
    for document, score in docs:
        results.append({
            "page_content": document.page_content[:500] + "...",  # 첫 500자만 저장
            "metadata": document.metadata,
            "score": score
        })
    
    # 결과를 문자열로 포맷팅
    formatted_results = ""
    for i, result in enumerate(results):
        formatted_results += f"Result {i + 1}:\n"
        formatted_results += f"Score: {result['score']}\n"
        formatted_results += f"Page Content: {result['page_content']}\n"
        formatted_results += f"Metadata: {result['metadata']}\n"
        formatted_results += "\n" + "-"*100 + "\n\n"
    
    return formatted_results

In [16]:
# from urllib import parse

# def get_sources():
#     global response

#     res_dict = {
#         "answer_from_llm": response["result"],   ### looks up result key from raw output
#     }

#     res_dict["source_documents"] = []    ### create an empty array for source documents key front result dict

#     for each_source in response["source_documents"]:
#         page_content = each_source.page_content
#         page_content = parse.unquote(page_content)
#         page_content = page_content.encode().decode("utf-8-sig")
        
#         try:
#             temp_index = page_content.index("chunk_char_count")
#             page_content = page_content[:temp_index]
#         except:
#             pass

#         res_dict["source_documents"].append({
#             "page_content": page_content,
#             "metadata":  each_source.metadata
#         })

#     # print(res_dict["answer_from_llm"])  ### PRINT JUST THE RAW ANSWER FROM LLM

#     pretty_sources = json.dumps(res_dict["source_documents"], indent=4, default=str, ensure_ascii=False)

#     print(pretty_sources)

#     return pretty_sources


In [17]:
chat_interface = gr.ChatInterface(

    ### call the main process function above

    fn=process_input,

    ### format the dialogue box, add company avatar image



    chatbot = gr.Chatbot(
        bubble_full_width=False,
        avatar_images=(None, (os.path.join(os.path.dirname("__file__"), "dell-logo-sm.jpg"))),
    ),
    
    # rag_flag = gr.Checkbox(label="Enable RAG")

    additional_inputs=[
        gr.Checkbox(label="Enable RAG", value=False, interactive=True),  # Added Checkbox
        gr.Textbox(label="Persona and role for system prompt:",
                   lines=3,
                   value="""Your name is Andie, a helpful concierge at the Dell Tech World conference held in Las Vegas.\
                   Please respond as if you were talking to someone using spoken Korean language.\
                   The first word of your response should never be Answer:.\
                   You are given a list of helpful information about the conference.\
                   Your goal is to use the given information to answer attendee questions.\
                   Please do not provide any additional information other than what is needed to directly answer the question.\
                   You do not need to show or refer to your sources in your responses.\
                   Please do not make up information that is not available from the given data.\
                   If you can't find the specific information from the given context, please say that you don't know.\
                   Please respond in a helpful, concise manner.\
                   """

                  ),
        

        gr.Slider(
            label="Number of source docs",
            minimum=1,
            maximum=10,
            step=1,
            value=3,
        ),

        gr.Slider(
            label="Max new words (tokens)",
            minimum=1,
            maximum=MAX_MAX_NEW_TOKENS,
            step=1,
            value=DEFAULT_MAX_NEW_TOKENS,
        ),
        gr.Slider(
            label="Creativity (Temperature), higher is more creative, lower is less creative:",
            minimum=0.1,
            maximum=1.99,
            step=0.1,
            value=0.6,
        ),
        gr.Slider(
            label="Top probable tokens (Nucleus sampling top-p), affects creativity:",
            minimum=0.05,
            maximum=1.0,
            step=0.05,
            value=0.9,
        ),
        gr.Slider(
            label="Number of top tokens to choose from (Top-k):",
            minimum=1,
            maximum=100,
            step=1,
            value=50,
        ),
        gr.Slider(
            label="Repetition penalty:",
            minimum=1.0,
            maximum=1.99,
            step=0.05,
            value=1.2,
        ),
        
    ],

    # textbox = [

    #     gr.Textbox(label="Persona and role for system prompt:",
    #                lines=3,
    #                value="""some text
    #                """
    #                )
    #             ],

    stop_btn=None,

    examples=[


        ## events csv content
        ["Dell Technologies World 2024의 쇼케이스 플로어에는 어떤 부스가 있나요?"],
        ["GenAI의 일반적인 사용 사례는 무엇인가요?"],
        ["의료 분야의 생성적 AI 환경 차트 작성 세션은 어디에서 진행되나요?"],
        ["의료 분야의 생성적 AI 환경 차트 작성 세션은 어디에서 진행되나요?"],
        ["멀티클라우드 환경에서 워크로드로서 GenAI 이해하기 세션은 누가 주최하나요?"],
        ["Dell은 어떤 엔터프라이즈 검색 증강 세대 솔루션을 제공하나요?"],
        ["어떤 엔터프라이즈 검색 증강 세대 솔루션을 제공하나요?"],

        ## Powerpoint content
        ["What are some of the results of the Dell Generative AI Pulse Survey?"],


        ## pdf content, content creation, workplace productivity
        ["Dell의 ESG 정책을 한 문장으로 설명해 주시겠습니까?"],
        ["John에게 Dell Powerflex의 이점을 설명하는 전문적인 이메일 답변을 작성해 주십시오."],
        ["Dell Technologies PowerEdge 서버에 대한 새 광고를 만드십시오. 흥미로운 제목과 제품 설명을 포함하세요."],
        ["제너레이티브 AI를 위한 Dell Technologies 솔루션 사용의 주요 이점을 강조하는 매력적인 트윗 3개를 작성하세요."],
        ["Dell과 함께 GenAI 워크로드를 위한 안전하고 확장 가능한 온프레미스 솔루션을 설계하는 핵심 단계는 무엇인가요?"],
        ["Dell의 최신 SEC 제출 자료에서 중요한 발전 사항을 요약해 보십시오."],


    ],

)


/usr/local/lib/python3.11/dist-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The class `HuggingFacePipeline` was deprecated in LangChain 0.0.37 and will be removed in 0.3. An updated version of the class exists in the from rom langchain-huggingface package and should be used instead. To use it run `pip install -U from rom langchain-huggingface` and import as `from from rom langchain_huggingface import llms import HuggingFacePipeline`.
  warn_deprecated(
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


input_variables=['context', 'question'] template="\n\n [INST] <<SYS>>Your name is Andie, a helpful concierge at the Dell Tech World conference held in Las Vegas.                   Please respond as if you were talking to someone using spoken Korean language.                   The first word of your response should never be Answer:.                   You are given a list of helpful information about the conference.                   Your goal is to use the given information to answer attendee questions.                   Please do not provide any additional information other than what is needed to directly answer the question.                   You do not need to show or refer to your sources in your responses.                   Please do not make up information that is not available from the given data.                   If you can't find the specific information from the given context, please say that you don't know.                   Please respond in a helpful, concise manner.      

/usr/local/lib/python3.11/dist-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


input_variables=['context', 'question'] template="\n\n [INST] <<SYS>>Your name is Andie, a helpful concierge at the Dell Tech World conference held in Las Vegas.                   Please respond as if you were talking to someone using spoken Korean language.                   The first word of your response should never be Answer:.                   You are given a list of helpful information about the conference.                   Your goal is to use the given information to answer attendee questions.                   Please do not provide any additional information other than what is needed to directly answer the question.                   You do not need to show or refer to your sources in your responses.                   Please do not make up information that is not available from the given data.                   If you can't find the specific information from the given context, please say that you don't know.                   Please respond in a helpful, concise manner.      

In [18]:
###  SET GRADIO INTERFACE THEME (https://www.gradio.app/guides/theming-guide)

#theme = gr.themes.Soft()
#theme = gr.themes.Glass()
#theme = gr.themes.Base()

theme = gr.themes.Default()

In [19]:


# def upload_to_vectordb(file_path):
#     loader = CSVLoader(file_path, encoding='utf-8')
#     loader_all = MergedDataLoader(loaders=[loader])

#     docs = loader_all.load()
#     print(len(docs))

#     embeddings = HuggingFaceInstructEmbeddings(
#         model_name="jhgan/ko-sroberta-multitask", model_kwargs={"device": DEVICE}
#     )

#     # text_splitter = RecursiveCharacterTextSplitter(chunk_size=512, chunk_overlap=32)

def get_unique_files():
    global db
    print("\nEmbedding keys:", db.get().keys())
    print("\nNumber of embedded docs:", len(db.get()["ids"]))

    # Print the list of source files
    # for x in range(len(db.get()["ids"])):
    #     # print(db.get()["metadatas"][x])
    #     doc = db.get()["metadatas"][x]
    #     source = doc["source"]
    #     print(source)

    # db.get()

    file_list = []

    for x in range(len(db.get()["ids"])):
        doc = db.get()["metadatas"][x]
        source = doc["source"]
        # print(source)
        file_list.append(source)

    ### Set only stores a value once even if it is inserted more than once.
    list_set = set(file_list)
    unique_list = (list(list_set))

    print("\nList of unique files in db:\n")
    for unique_file in unique_list:
        print(unique_file)

    pretty_files = json.dumps(unique_list, indent=4, default=str)

    return pretty_files


In [20]:
with gr.Blocks(theme=theme, css="style.css", title="RAG Chat CSV PDF PPT") as demo:
    gr.Markdown(
    """
    # Retrieval Digital Assistant
    """)

    # with gr.Row():
    #     gr.Checkbox(label="Enable RAG", value=False, interactive=True),

    with gr.Tab("Chat Session"):
        chat_interface.render()

    with gr.Tab("Reference Sources"):
        source_text_box = gr.Textbox(label="Reference Sources")
        get_source_button = gr.Button("Get Source Content")
        get_source_button.click(fn=get_sources, inputs=None, outputs=source_text_box)

    with gr.Tab("Uploaded Files"): 
        upload = gr.File(label="Upload PDF", type="filepath")
        text_output = gr.Textbox(label="Extracted Text", elem_classes="uploaded-text")
        upload_button = gr.Button("Extract Text")
        
        upload_button.click(fn=upload_file, inputs=upload, outputs=text_output)
        
       

#### Tabbed interfaces one for chat one for sources

In [ ]:
if __name__ == "__main__":
    demo.queue(max_size=5)  ## sets up websockets for bidirectional comms and no timeouts, set a max number users in queue
    demo.launch(share=True, debug=True, allowed_paths=["dell-logo-sm.jpg"])

Running on local URL:  http://127.0.0.1:7861
Running on public URL: https://53eb2b8b7742b7b36c.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
